In [1]:
# Packages to be used

import numpy as np
import pandas as pd
import glob
import time
import datetime

from PIL import ImageFilter, ImageStat, Image, ImageDraw
import matplotlib.pyplot as plt
import seaborn as sns
#%matplotlib inline
import cv2

from multiprocessing import Pool, cpu_count

from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV, StratifiedKFold, cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import log_loss
from sklearn.utils import class_weight

from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Flatten
from keras.layers.convolutional import Convolution2D, ZeroPadding2D, MaxPooling2D
from keras.optimizers import SGD, Adam, RMSprop, Nadam, Adagrad, Adamax, TFOptimizer, Adadelta, TFOptimizer
from keras import optimizers
from keras.utils import np_utils
from keras import backend as K
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard 
from keras.regularizers import l2, activity_l2, l1, l1l2, activity_l1, activity_l1l2 

from keras import __version__ as keras_version             


Using TensorFlow backend.


In [3]:
# Data loading and shuffling
train_data = np.load('../deep/cancer/inputs/train.npy')
train_target = np.load('../deep/cancer/inputs/train_target4.npy')
train_id = np.load('../deep/cancer/inputs/train_id2.npy')
test_data = np.load('../deep/cancer/inputs/test2.npy')
test_id = np.load('../deep/cancer/inputs/test_id.npy')
train_target = np_utils.to_categorical(train_target, 3)

train_data = shuffle(train_data)
train_target = shuffle(train_target)
test_data = shuffle(test_data)

print(len(train_data))
print(len(train_target))
print(len(test_data))

8134
8134
512


In [4]:
# parameters used in model 
early_stopping = EarlyStopping(monitor='val_loss', patience=20)
now = datetime.datetime.now()
checkpointer = ModelCheckpoint(filepath="weights/"'checkpointer_' + '_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.hdf5', verbose=1, save_best_only=True)
#class_weight = class_weight.compute_class_weight('balanced', np.unique(train_target), train_target)

### image dimensions
num_channels = 3
imageSize = (64, 64)
img_width, img_height = imageSize[0], imageSize[1]


In [ ]:
# Actual
def create_model():
    wr1 = 1e-04
    wr2 = 1e-04
    wr3 = 1e-03
    wr4 = 1e-03
    wr5 = 1e-03

    activation = 'relu'
    optimizer = 'sgd'
     
    model = Sequential()
    model.add(ZeroPadding2D((1, 1), input_shape=(num_channels, img_width, img_height), dim_ordering='tf'))
    model.add(Convolution2D(64, 3, 3, activation=activation, dim_ordering='tf', W_regularizer=l1l2(wr1), activity_regularizer=activity_l2(wr1), init='lecun_uniform'))
    #model.add(ZeroPadding2D((1, 1), dim_ordering='tf'))
    #model.add(Convolution2D(32, 3, 3, activation=activation, dim_ordering='tf', W_regularizer=l1l2(wr1), activity_regularizer=activity_l2(wr1), init='lecun_uniform'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), dim_ordering='tf'))
    model.add(Dropout(0.85))

    """
    model.add(ZeroPadding2D((1, 1), dim_ordering='tf'))
    model.add(Convolution2D(64, 3, 3, activation=activation, dim_ordering='tf', W_regularizer=l1l2(wr2), activity_regularizer=activity_l2(wr2), init='lecun_uniform'))
    model.add(ZeroPadding2D((1, 1), dim_ordering='tf'))
    model.add(Convolution2D(128, 3, 3, activation=activation, dim_ordering='tf', W_regularizer=l1l2(wr2), activity_regularizer=activity_l2(wr2), init='lecun_uniform'))
    #model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), dim_ordering='tf'))
    model.add(Dropout(0.85))

    model.add(ZeroPadding2D((1, 1), dim_ordering='tf'))
    model.add(Convolution2D(64, 3, 3, activation=activation, dim_ordering='tf', W_regularizer=l1(wr3), activity_regularizer=activity_l1(wr3), init='lecun_uniform'))
    model.add(ZeroPadding2D((1, 1), dim_ordering='tf'))
    model.add(Convolution2D(64, 3, 3, activation=activation, dim_ordering='tf', W_regularizer=l1(wr3), activity_regularizer=activity_l1(wr3), init='lecun_uniform'))
    model.add(ZeroPadding2D((1, 1), dim_ordering='tf'))
    model.add(Convolution2D(128, 3, 3, activation=activation, dim_ordering='tf', W_regularizer=l1(wr3), activity_regularizer=activity_l1(wr3), init='lecun_uniform'))
    #model.add(MaxPooling2D((2, 2), strides=(2, 2), dim_ordering='tf'))
    model.add(Dropout(0.85))
    
    model.add(ZeroPadding2D((1, 1), dim_ordering='tf'))
    model.add(Convolution2D(128, 3, 3, activation=activation, dim_ordering='tf', W_regularizer=l1(wr4), activity_regularizer=activity_l1(wr4), init='lecun_uniform'))
    model.add(ZeroPadding2D((1, 1), dim_ordering='tf'))
    model.add(Convolution2D(256, 3, 3, activation=activation, dim_ordering='tf', W_regularizer=l1(wr4), activity_regularizer=activity_l1(wr4), init='lecun_uniform'))
    model.add(ZeroPadding2D((1, 1), dim_ordering='tf'))
    model.add(Convolution2D(256, 3, 3, activation=activation, dim_ordering='tf', W_regularizer=l1(wr4), activity_regularizer=activity_l1(wr4), init='lecun_uniform'))
    #model.add(MaxPooling2D((2, 2), strides=(2, 2), dim_ordering='tf'))
    model.add(Dropout(0.95))

    model.add(ZeroPadding2D((1, 1), dim_ordering='tf'))
    model.add(Convolution2D(128, 3, 3, activation=activation, dim_ordering='tf', W_regularizer=l1(wr5), activity_regularizer=activity_l1(wr5), init='lecun_uniform'))
    model.add(ZeroPadding2D((1, 1), dim_ordering='tf'))
    model.add(Convolution2D(256, 3, 3, activation=activation, dim_ordering='tf', W_regularizer=l1(wr5), activity_regularizer=activity_l1(wr5), init='lecun_uniform'))
    model.add(ZeroPadding2D((1, 1), dim_ordering='tf'))
    model.add(Convolution2D(256, 3, 3, activation=activation, dim_ordering='tf', W_regularizer=l1(wr5), activity_regularizer=activity_l1(wr5), init='lecun_uniform'))
    #model.add(MaxPooling2D((2, 2), strides=(2, 2), dim_ordering='tf'))
    model.add(Dropout(0.95))
    """
    
    model.add(Flatten())

    #model.add(Dense(8, activation=activation, W_regularizer=l2(wr4), init='lecun_uniform'))
    #model.add(Dropout(0.5))
    #model.add(Dense(256, activation=activation, W_regularizer=l2(wr4), init='lecun_uniform'))
    #model.add(Dropout(0.5))
    #model.add(Dense(128, activation=activation, W_regularizer=l2(wr5), init='lecun_uniform'))
    #model.add(Dropout(0.5))
    #model.add(Dense(128, activation=activation, W_regularizer=l2(wr5), init='lecun_uniform'))
    #model.add(Dropout(0.5))
  
    model.add(Dense(3, activation='softmax'))

    sgd = SGD(lr=1e-4, momentum=0.9, clipvalue=0.5, nesterov=True)
    adam = Adam(lr=1e-4)
    nadam = Nadam(lr=1e-2)
    adagrad = Adagrad(lr=1e-2)
    adamax = Adamax(lr=1e-1)
    adadelta = Adadelta(lr=10)

    model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])
    
    model.summary()
    
    model_result = model.fit(train_data, train_target, 
             nb_epoch=100, 
             validation_split=0.5, 
             shuffle=True, 
             batch_size=32,
             class_weight = 'auto',
             callbacks=[early_stopping, checkpointer], verbose=1)

    return model_result
